<a href="https://colab.research.google.com/github/Jaejuna/SpeedWagon/blob/main/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/train.csv')
valid = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid.csv')

In [5]:
train.head()

,Text,Summary,Topic
0,웃긴게 20년동안... 바로 옆 가게 주차장에서 같이 그 감자탕집 주차도 했었는데 ...,20년 동안 감자탕 집 주차도 했었는데 감자탕 집이 장사 잘 되니까 배 아파서 주차...,주거와생활
1,아까 보니깐 로켓 프레시 잇드라 #@시스템#사진# 나도 밥 무거요 호앵 야 제주도는...,제주시에 쿠팡 로켓 배송이나 택배가 오는지 물어보고 있다.,주거와생활
2,#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면...,이불 위에 머리 망이 없다고 하니까 그럼 넣은 것 같다고 했다.,주거와생활
3,분양가 얼마여써? 여기 일억육천구백이더라ㅔ ㅋㅋㅋㅋㅋ걍 여기서 살고싶다..........,분양가 이억이 넘는데 일억은 대출로 받아야 한다.,주거와생활
4,지금 뭔가 카톡 느리지않녀 새해라 다들 톡하느라 터진건가 ㅇㅇ 약간 버벅댐 ㅇㅇㅇ ...,새해에 다들 카톡을 해서 그런지 카톡이 약간 느렸지만 괜찮아졌다.,주거와생활


In [6]:
valid.head()

,Text,Summary,Topic
0,수유로오셈^^ 지금? 지금가도대? 지금이겠냐? 되겠냐? 집 데려다주냐? 왜안댐? ㅋ...,지금 수유에 갈 테니 집에 데려다 달라고 한다.,행사
1,제주도가구시퍼져써 여보시간 업자나 2박3일정두 갔다올수이ㅛ을거같운데 쉬는날몰아서 구...,2박 3일 정도 쉬는 날을 몰아서 제주도에 갔다 오기로 했다.,행사
2,무튼 몇시에 어디서 만나지? 정한거 아냐? 메가박스 2시 45분 매표소?역출구? 매...,메가박스 매표소에서 2시 45분에 만나기로 약속했다.,행사
3,낼상황봐서 정하자 오켕 낼 보구 밥 정하쟈 ~ 웅 조아용 그리구 간지카페가자 힐링해...,내일 상황을 봐서 정하자고 카페를 가서 힐링해야 한다고 한다.,행사
4,내일 모레 머먹어? 김장김치랑 수육!!!!!!!!!! 수!!! 육!!!!! 나는? ...,모레 김장 김치와 수육을 먹을 테니 집으로 오라고 한다.,행사


## EDA

In [ ]:
print("Size of train data :", len(train))
print("Size of validation data :", len(valid))

Size of train data : 279992
Size of validation data : 35004


In [ ]:
# Train / Valid dataset split rate
len(train) / len(valid) # 8:1 비율

7.998857273454463

In [ ]:
# Topic 종류
train['Topic'].unique()

array(['주거와생활', '시사교육', '상거래(쇼핑)', '개인및관계',
       '행사', '미용과건강', '일과직업', '식음료', '여가생활'],
      dtype=object)

In [ ]:
valid['Topic'].unique()

array(['행사', '시사교육', '상거래(쇼핑)', '개인및관계',
       '주거와생활', '여가생활', '미용과건강', '일과직업',
       '식음료'], dtype=object)

In [ ]:
# Topic 별 Dialogue 수
pd.DataFrame(train.groupby("Topic").count().iloc[:,0])

,Text
Topic,
개인및관계,71130
미용과건강,17069
상거래(쇼핑),26298
시사교육,13119
식음료,30184
여가생활,35247
일과직업,20428
주거와생활,45179
행사,21338


In [ ]:
pd.DataFrame(valid.groupby("Topic").count().iloc[:,0])

,Text
Topic,
개인및관계,8892
미용과건강,2134
상거래(쇼핑),3288
시사교육,1640
식음료,3773
여가생활,4407
일과직업,2554
주거와생활,5648
행사,2668


## Data Preprocessing

In [7]:
# 네이버 맞춤법 검사기
!pip install git+https://github.com/jungin500/py-hanspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jungin500/py-hanspell to /tmp/pip-req-build-bnzd9_5w
  Running command git clone --filter=blob:none --quiet https://github.com/jungin500/py-hanspell /tmp/pip-req-build-bnzd9_5w
  Resolved https://github.com/jungin500/py-hanspell to commit b4fb712334496821db4dffb981cc73a9bbaee673
  Preparing metadata (setup.py) ... done


In [8]:
from hanspell import spell_checker
import re

In [10]:
def preprocess(text):
  text = text.lower() # 영어 텍스트를 소문자로 변환
  text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', text) # 반복되는 자음, 모음 제거 (ex. ㅋㅋ ㅠㅠ)
  text = spell_checker.check(text).checked # 맞춤법 검사
  text = re.sub(r'\.{3,}', '..', text) # .이 3개 이상 반복되면 2개로 변환 
  text = re.sub(r'\?{2,}', '?', text) # ?이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'\!{2,}', '!', text) # !이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'\~{2,}', '~', text) # ~이 2개 이상 반복되면 1개로 변환
  text = re.sub(r'[" "]+', " ", text) # 공백 여러 개를 1개로 변환
  text = text.strip() # 양쪽 공백 제거
  return text

In [11]:
train.iloc[2,0]

'#@이름#아 먼가 나머리망 이불위에 두고 온거 같다 확인좀 ㅠ 읭?? 만약에 잇으면 침대 밑 바닥에 놔줭 #@이모티콘# 기달 업는뎅ㅜ 구럼 내가 너엇나바 ㅋㅋㅋㅋ 땡큐'

In [18]:
preprocess(train.iloc[2,0])

'#@이름#아 먼가 나 머리망 이불 위에 두고 온 거 같다 확인 좀 ㅠ 응? 만약에 있으면 침대 밑 바닥에 놔줘 #@이모티콘# 기달 업는데ᅮ 그럼 내가 너 엇나 바 땡큐'

In [ ]:
for i in range(train.shape[0]):
  try:
    train.iloc[i,0] = preprocess(train.iloc[i,0])
  except:
    print(i, "번째 error")

In [12]:
for i in range(valid.shape[0]):
  try:
    valid.iloc[i,0] = preprocess(valid.iloc[i,0])
  except:
    print(i, "번째 error")

1336 번째 error
6206 번째 error
7125 번째 error
9014 번째 error
11606 번째 error
11639 번째 error
13517 번째 error
14660 번째 error
19434 번째 error
19552 번째 error
21310 번째 error
21670 번째 error
24859 번째 error
26398 번째 error
28204 번째 error
29665 번째 error
32211 번째 error
32457 번째 error
32938 번째 error
34112 번째 error
34428 번째 error


## 오류 처리

- 오류 발생 데이터들

  => & 기호로 인해 오류 발생

  => 이모티콘으로 오류 발생

In [21]:
valid.iloc[1336,0]

'그나저나 내일은 머하지....... 머해ㅠㅠㅠㅠㅠㅠㅠㅠ 그냥 같이 집에서 쉬고싶다ㅠㅠㅠㅠ 넷플릭스 보고 맛난거 먹고ㅠㅠㅠㅠㅠㅠㅠㅠ 아파트면,...집에서 보내는 것도 괜츈한데...ㅋㅋㅋㅋ윽... 힝..ㅠㅠㅠ 아니면 동해 갈까?~ 어차피 이대로 계획 없으면 내일 동네 카페 갈 것이 불을 보듯 뻔한 일이니 동해시에 묵호등대 & 벽화마을도 있고ㅋㅋ 동해도 매진 된거 아냐 이미? 당장 내일인데 동해는 버스 있어~ㅋ 이게 사람들이 동해시는 잘 모르더라고ㅋㅋ'

In [20]:
valid.iloc[6206,0]

'오늘 펜션 취소했는데 국도로 못간다고 했더니 고속도로로 오라던 주인.. 저기 저 고속도로 지나서 가야하는데 저기도 붕괴됐구요 어우 결국 환불 못받고 날짜 옮김 비수기로.. ㅡㅡ 돈에 미쳐5:&'

In [23]:
preprocess(valid.iloc[6206,0].replace('&',""))

'오늘 펜션 취소했는데 국도로 못 간다고 했더니 고속도로로 오라던 주인.. 저기 저 고속도로 지나서 가야 하는데 저기도 붕괴됐고요 어우 결국 환불 못 받고 날짜 옮김 비수기로.. 돈에 미쳐 5:'

In [25]:
valid.iloc[11606,0]

'자긴 밥먹었어요??❤️❤️ 쟈기 ㅠㅠ 정신없엇지 ㅠㅠㅠ 11시반에 먹고 왓어요 ㅎㅎㅎ 자기 공연은 몇시에요?!💗 자기 먹구왓구나!! 7시인가?? 가서 물어보려구요 ㅠㅠ 난 이것도 모르고있네 ㅠㅠ 자기 사랑해❤️ 나는 다시 수업 듣고올게요 내사랑💕💕 쟈기 사랑해요❤️ㅎㅎ 오늘 따라 더더더 보고싶어요 🥰내사랑 조금만 더 고생해요 내꾸 ㅜㅠ 잘다녀와🧡💙💗 나두요ㅠㅠㅠ 너무보고싶다....😂😂😂😂 내사랑 일요일까지 조금만 참을려구요ㅠㅠㅠ'

In [26]:
valid.iloc[11639,0]

'애드라~~ 새해복많이받구 우리 2019년에는 좋은일만 있자💕 새해복많이받앙!! #@시스템#사진# 해피뉴이어.. 예쁘다 다들 사랑해🐌💕✨❣️❣️❣️❣️🤗😒😒😒😒🐌🐌🐌🤗🤗🤗🤗🤗🤗😀😀😀😀😀😀 해피뉴이어 미쳐진짴ㅋㅌㅋㅋㅌㅌㅋㅋㅋㅋ 히히히 새해복 마니받아'

In [27]:
valid.iloc[19552,0]

'여기너무추워나감기든가같애 눈내렸어 와우 역시강원도고만 진짜춥ㄷ 기속사 그거틀어 개꾸짐 히터 ㅋㅋㅋㅋ 히텂어 어이구 그럼뭐야 보일러인가 더좋네 놉 바닥도안들어옴 그거알아? 우리여중때 아 창가쪽에 헐 와우 그거임 그거진짜 그립다 개취워진심 옷다껴입고 ㅋㅋㅋㅋㅋㅋㅋ 힘내'

In [33]:
valid.iloc[11606,0] = valid.iloc[11606,0].replace("🧡💙💗","").replace("❤️","").replace("💗","").replace("😂","").replace("💕","").replace("🥰","")

In [29]:
valid.iloc[11639,0] = valid.iloc[11639,0].replace("🐌💕✨❣️❣️❣️❣️🤗😒😒😒😒🐌🐌🐌🤗🤗🤗🤗🤗🤗😀😀😀😀😀😀","").replace("💕","")

In [35]:
error_list = [1336, 6206, 7125, 9014, 11606, 11639, 13517, 14660, 19434, 19552,21310, 21670, 24859, 26398, 28204, 29665, 32211, 32457, 32938, 34112, 34428]

for i in error_list:
  try:
    valid.iloc[i,0] = preprocess(valid.iloc[i,0].replace('&',""))
  except:
    print(i, "번째 error")

19552 번째 error


## 전처리 파일 저장

In [ ]:
train.head()

In [13]:
valid.head()

,Text,Summary,Topic
0,수유로 오세요^^ 지금? 지금 가도 대? 지금이겠냐? 되겠냐? 집 데려다주냐? 왜 ...,지금 수유에 갈 테니 집에 데려다 달라고 한다.,행사
1,제주도가 구 시퍼죠 써 여보 시간 업자나 2박 3일 정도 갔다 올 수 이ㅛ을 거 같...,2박 3일 정도 쉬는 날을 몰아서 제주도에 갔다 오기로 했다.,행사
2,아무튼 몇 시에 어디서 만나지? 정한 거 아냐? 메가박스 2시 45분 매표소? 역 ...,메가박스 매표소에서 2시 45분에 만나기로 약속했다.,행사
3,낼 상황 봐서 정하자 오케이 낼 보고 밥 정하자 ~ 웅 좋아요 그리고 간지 카페가 ...,내일 상황을 봐서 정하자고 카페를 가서 힐링해야 한다고 한다.,행사
4,내일모레 뭐 먹어? 김장김치랑 수육! 수! 육! 나는? 너도 와 그럼 나도 수육.....,모레 김장 김치와 수육을 먹을 테니 집으로 오라고 한다.,행사


In [ ]:
train.to_csv('/content/drive/MyDrive/BOAZ_miniProject2/train_전처리.csv', index = False)

In [36]:
valid.to_csv('/content/drive/MyDrive/BOAZ_miniProject2/valid_전처리.csv', index = False)

https://velog.io/@jx7789/%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1-%EB%8C%80%ED%99%94-%EC%9A%94%EC%95%BD-%EC%84%9C%EB%B9%84%EC%8A%A4-pepkrzfe